## Machine Learning ways to do classification


In [ ]:
#Data Preprocessing 
y = data['problem_type']
X = data['situation']
length = data['situation'].apply(len)
data = data.assign(Len_Situation=length)

# Split train & test
text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)
X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set
print('vocabubary size:',vocab_size)
print('max length text:',maxlen)

#Padding the sentences
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

#embedding dimension should align with the GLOVE
embedding_dim = 100
vocab_size = len(word_index) + 1

GLOVE_6B_100D_PATH ="/content/drive/MyDrive/PLP_SharedFolder/User_Condition_Classification_model/glove.6B.100d.txt"
encoding="utf-8"
with open(GLOVE_6B_100D_PATH, "rb") as lines:
    embeddings_index = {line.split()[0].decode(encoding): np.array(line.split()[1:],dtype=np.float32)
               for line in lines}

# Prepare embedding matrix from pre-trained model
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# Check % words with embeddings 
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
print(nonzero_elements / vocab_size)   

vocabubary size: 2450
max length text: 196
0.9575510204081633


In [ ]:
!pip install nltk
import nltk
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn import metrics 
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
#Data Preprocessing 
y = data['problem_type']
X = data['situation']
length = data['situation'].apply(len)
data = data.assign(Len_Situation=length)

#Split train & test
text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

text_clf_NB = Pipeline([('vect', CountVectorizer()),   #Vectorizer
                     ('tfidf', TfidfTransformer()), #DTM with TFIDF
                      ('clf', RidgeClassifier()),     #ML Model
                    ])
text_clf_NB.fit(text_train,y_train ) 

##Evaluate the model
print("Performance on Test Data")
predicted_0 = text_clf_NB.predict(text_test)
print("Ridge:",np.mean(predicted_0 == y_test) )

print("Overall Performance")
predicted_1 = text_clf_NB.predict(X)
print("Ridgeclassifier:",np.mean(predicted_1 == y) )

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

text_clf_NB = Pipeline([('vect', CountVectorizer()),   #Vectorizer
                     ('tfidf', TfidfTransformer()), #DTM with TFIDF
                      ('clf', RandonForestClassifier(n_estimators=2000)),     #ML Model
                    ])
text_clf_NB.fit(text_train,y_train ) 

##Evaluate the model
print("Performance on Test Data")
predicted_0 = text_clf_NB.predict(text_test)
print("RF:",np.mean(predicted_0 == y_test) )

print("Overall Performance")
predicted_1 = text_clf_NB.predict(X)
print("RF:",np.mean(predicted_1 == y) )

In [ ]:
#####################Build training pipeline using  Naïve Bayes Model

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=2000, random_state=1)
clf3 = RidgeClassifier()
clf4=svm.LinearSVC(C=1.0)
text_clf_NB = Pipeline([('vect', CountVectorizer()),   #Vectorizer
                     ('tfidf', TfidfTransformer()), #DTM with TFIDF
                      ('clf3', VotingClassifier(estimators=[
               ('rf', clf2), ('gnb', clf3),('svm',clf4)],voting='hard')),     #ML Model
                    ])


text_clf_NB.fit(text_train,y_train ) 

##Evaluate the model
print("Performance on Test Data")
predicted_0 = text_clf_NB.predict(text_test)
print("VC:",np.mean(predicted_0 == y_test) )

print("Overall Performance")
predicted_1 = text_clf_NB.predict(X)
print("votingclassifier:",np.mean(predicted_1 == y) )

Performance on Test Data
VC: 0.7666666666666667
Overall Performance
votingclassifier: 0.9276923076923077
